# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [2]:
# Drop unnecessary columns
new_df = df.drop('timestamp', axis = 1)

It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [3]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 
read = Reader()
data = Dataset.load_from_df(new_df[['userId','movieId', 'rating']], read)



Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [4]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)


Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [5]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [15]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
param_grid = {'n_factors':[10,30,50,60,40],
             'reg_all':[0.01, 0.05]}
svd_model = SVD()
grid_model = GridSearchCV(SVD,param_grid = param_grid, n_jobs = -1, joblib_verbose = 5,measures = ['rmse','mae'],refit = False)
grid_model.fit(data)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 out of  50 | elapsed:    7.6s remaining:    2.3s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    8.6s finished


In [19]:
# print out optimal parameters for SVD after GridSearch
print('best parameters rmse :',grid_model.best_params['rmse'])
print('best parameters mae :',grid_model.best_params['mae'])
print()
print('best score rmse:',grid_model.best_score['rmse'])
print('best score mae:',grid_model.best_score['mae'])

best parameters rmse : {'n_factors': 40, 'reg_all': 0.05}
best parameters mae : {'n_factors': 40, 'reg_all': 0.05}

best score rmse: 0.8696690861165784
best score mae: 0.6685350679722746


In [21]:
# cross validating with KNNBasic
knns = KNNBasic()
cross_validate(knns,data,measures = ['rmse','mae'], cv = 5,verbose = True)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9477  0.9439  0.9495  0.9542  0.9364  0.9463  0.0060  
MAE (testset)     0.7256  0.7221  0.7282  0.7319  0.7195  0.7255  0.0044  
Fit time          0.27    0.30    0.31    0.26    0.31    0.29    0.02    
Test time         1.70    1.77    1.63    1.90    1.66    1.73    0.10    


{'test_rmse': array([0.94768201, 0.94391546, 0.94947922, 0.95416403, 0.93641403]),
 'test_mae': array([0.72555196, 0.72210278, 0.72817262, 0.73189735, 0.71953304]),
 'fit_time': (0.2658112049102783,
  0.30399465560913086,
  0.3071146011352539,
  0.2626221179962158,
  0.3106234073638916),
 'test_time': (1.697619915008545,
  1.7684438228607178,
  1.627119779586792,
  1.8967094421386719,
  1.6600806713104248)}

In [24]:
# print out the average RMSE score for the test set
results = cross_validate(knns,data,measures = ['rmse','mae'], cv = 5,verbose = True)
average_rmse = results['test_rmse'].mean()
print(f'average rmse:{average_rmse}')


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9388  0.9513  0.9459  0.9522  0.9446  0.9466  0.0049  
MAE (testset)     0.7187  0.7298  0.7265  0.7295  0.7250  0.7259  0.0040  
Fit time          0.29    0.12    0.19    0.12    0.12    0.17    0.06    
Test time         1.48    0.87    0.94    0.90    0.94    1.03    0.23    
average rmse:0.9465703254914029


In [25]:
# cross validating with KNNBaseline
knn_baseline = KNNBaseline()
results_baseline = cross_validate(knn_baseline,data,measures = ['rmse','mae'], cv = 5,verbose = True)


Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8799  0.8718  0.8695  0.8762  0.8748  0.8744  0.0036  
MAE (testset)     0.6738  0.6687  0.6637  0.6677  0.6691  0.6686  0.0032  
Fit time          0.28    0.28    0.45    0.42    0.24    0.33    0.09    
Test time         1.13    1.46    1.36    1.23    1.15    1.27    0.13    


In [26]:
# print out the average score for the test set
average_rmse_baseline = results_baseline['test_rmse'].mean()
print(f'average rmse baseline:{average_rmse}')


average rmse baseline:0.9465703254914029


Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [29]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [30]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [20]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [21]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=3.0129484092252135, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [48]:
import random
def movie_rater(df_movies,num, genre=None):
    if genre:
        movie_df = df_movies[df_movies['genres'].str.contains(genre, case=False, na=False)]
        selected_movies = df_movies.sample(n=num, replace=True)
    
   
    rating_list = []
    for index, row in selected_movies.iterrows():
        user_id = random.randint(1, 100)  # Random userId for demonstration
        rating = round(random.uniform(1, 5), 1)  # Random rating between 1 and 5
        rating_list.append({'userId': user_id, 'movieId': row['movieId'], 'rating': rating})
    
    return rating_list

In [51]:
# try out the new function here!
user_rating = movie_rater(df_movies,4,'Comedy')

If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [52]:
user_rating

[{'userId': 24, 'movieId': 961, 'rating': 4.1},
 {'userId': 92, 'movieId': 49274, 'rating': 2.5},
 {'userId': 94, 'movieId': 27370, 'rating': 4.2},
 {'userId': 9, 'movieId': 6852, 'rating': 4.8}]

In [56]:
ratings = []
rating = [entry['rating'] for entry in user_rating]
ratings.append(rating)
ratings

[[4.1, 2.5, 4.2, 4.8]]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [58]:
## add the new ratings to the original ratings DataFrame
df['ratings'] = rating_list

NameError: name 'rating_list' is not defined

In [40]:
# train a model using the new combined DataFrame


In [42]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)


In [45]:
# order the predictions from highest to lowest rated

ranked_movies = None

 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [49]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
        pass
            
recommended_movies(ranked_movies,df_movies,5)

Recommendation #  1 :  277    Shawshank Redemption, The (1994)
Name: title, dtype: object 

Recommendation #  2 :  680    Philadelphia Story, The (1940)
Name: title, dtype: object 

Recommendation #  3 :  686    Rear Window (1954)
Name: title, dtype: object 

Recommendation #  4 :  602    Dr. Strangelove or: How I Learned to Stop Worr...
Name: title, dtype: object 

Recommendation #  5 :  926    Amadeus (1984)
Name: title, dtype: object 



## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.